**Импорт Пакетов функций**



In [1]:
#@title
import numpy as np
import math
import cmath
from functools import reduce
from copy import deepcopy
import matplotlib.pyplot as plt
import numba as nb

ModuleNotFoundError: No module named 'numpy'

**Функции квантовых гейтов**

In [26]:
#@title
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]], dtype = 'complex_')
X = np.array([[0,1],[1,0]], dtype = 'complex_')
I = np.array([[1,0],[0,1]], dtype = 'complex_')
T = np.array([[1,0],[0, cmath.exp((1j*cmath.pi)/4)]], dtype = 'complex_')
T_ermit = np.array([[1,0],[0, cmath.exp((-1j*cmath.pi)/4)]], dtype = 'complex_')
V = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1+1j)/2,(1-1j)/2],[0,0,(1-1j)/2,(1+1j)/2]], dtype = 'complex_')
V_ermit = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1-1j)/2,(1+1j)/2],[0,0,(1+1j)/2,(1-1j)/2]], dtype = 'complex_')
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype = 'complex_')
Zgate = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]], dtype = 'complex_')
Toffoli = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 1, 0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 0, 0, 0],
           [0, 0, 0, 0, 0, 1, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 1],
           [0, 0, 0, 0, 0, 0, 1, 0]],dtype = 'complex_')




def UnitGate(q, matrix, q0):
  qubit = deepcopy(q)
  for i in range(len(qubit)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r = 0
    else:
      r = 1
    qubit[i] = q[i|(1<<q0)] * matrix[r][1] + q[(i|(1<<q0))^(1<<q0)] * matrix[r][0]
  return qubit

#if you need to impose some kind of control operation NOT ,
# then you need to write (-) and the desired number. If it is 0, you need to
# write the same number in place of the control qubit as the controlled one

def UnitGatetwo(q,matrix,q1,q0):
  qubit = deepcopy(q)
  if q1 >= 0 and q1 != q0:
    for i in range(len(q)):
      if (i % (1<<(q0+1))) < (1<<(q0)):
        r1 = 0
      else:
        r1 = 1
      if (i %(1<<(q1+1))) < (1<<(q1)):
        r2 = 0
      else:
        r2 = 1
      qubit[i] = q[(i|(1<<q0))|(1<<q1)] * matrix[2*r2 +r1][3]  \
      + q[((i|(1<<q0))|(1<<q1))^(1<<q0)] * matrix[2*r2 +r1][2] \
      + q[((i|(1<<q0))|(1<<q1))^(1<<q1)] * matrix[2*r2 +r1][1] \
      + q[(((i|(1<<q0)|(1<<q1))^(1<<q1))^(1<<q0))] * matrix[2*r2 +r1][0]
  elif q1 == q0:
    qubit = UnitGate(qubit,X,0)
    qubit = UnitGatetwo(qubit,matrix,0,q0)
    qubit = UnitGate(qubit,X,0)
  else:
    qubit = UnitGate(qubit,X,-q1)
    qubit = UnitGatetwo(qubit,matrix,-q1,q0)
    qubit = UnitGate(qubit,X,-q1)

  return qubit


# Translation of the i qubit by count into a bit record of size n
# Input (n) is the number of qubits, (i) is the number of qubit 
# output int bit record 
def inttobit(n, i):
  return list(map(int, list(format(i, "0{0}b".format(n)))))

def powcomplex(complexmatrix):
  return (complexmatrix * (complexmatrix.real - 1j*complexmatrix.imag)).real

In [ ]:
#@title
n = int(input())
q = np.zeros((2**(n)), dtype = 'complex_')
q[3] = 1
print(q)
UnitGatetwo(q,CNOT,1,1)

**QFT** 



In [4]:
#@title
def CROT(n):
  return np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,cmath.exp((1j*cmath.pi)/2**(n-1))]])

def CROTminus(n):
  return np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,cmath.exp((-1j*cmath.pi)/2**(n-1))]])  


SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]], dtype = 'complex_')
def SWAPQFT(qubit, n):
  for i in range(n//2 ):
    qubit = UnitGatetwo(qubit, SWAP, i, n-i-1)
  return qubit

def QFT1(qubit,n):
  if n == 0:
    return qubit
  else:
    qubit = UnitGate(qubit,H,n-1)
    if n != 1:
      for i in range(n-1):
        qubit = UnitGatetwo(qubit, CROT(n-i),i ,n-1)
    return QFT1(qubit,n-1)

def QFT(qubit,n):
  return(SWAPQFT(QFT1(qubit,n), n))


def ff(qubit, n, k):
  if k == n+1:
    return qubit
  else:
    if k != 1:
      for i in range(k-1):
        qubit = UnitGatetwo(qubit, CROTminus(2+i),k-2-i ,k-1)
    qubit = UnitGate(qubit,H,k-1)
    return ff(qubit,n,k+1)


def inversQFT(qubit,n):
  return ff((SWAPQFT(qubit,n)),n,1)

In [ ]:
#@title
n = int(input())
q = np.zeros((2**(n)),dtype = 'complex_')
condition = int(input())
q[condition] = 1
print(q)
qubitQFT = QFT(q,n)
print(qubitQFT)
print(inversQFT(qubitQFT,n))

**QPE**

In [3]:
#@title
def Pgate(alphaint):
  return np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,cmath.exp((1j* cmath.pi)/alphaint)]])

def phase_cheat(qubit,alphaint,n):
  cycle = 1
  for i in range(n):
    for j in range(cycle):
      qubit = UnitGatetwo(qubit, Pgate(alphaint),i ,n)
    cycle = 2 * cycle
  return qubit

For T - operator

output - the general phase of the operator

In [ ]:
#@title
n = int(input())
q = np.zeros((2**(n+1)),dtype = 'complex_')
q[1<<n] = 1
for i in range((n)):
  q = UnitGate(q,H, i)

# exp(2 * i *pi \theta ) - operator phase
# exp(i*pi / phi )

phi = float(input())
q2 = phase_cheat(q, phi, n)
q3 = powcomplex(inversQFT(q2,n))
indexnonzero = np.argmax(q3)
bit = (bin(indexnonzero)[3:])
print(int(bit,2),'/',2**n)